In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from db_utils import sp2df, df2db, sp_run
from astro_utils import mjd_to_datetime
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_mjds, integrate_mjds_df
from ra_dec import calc_topos
from ztf_data import calc_ztf_detection_times, ztf_raw_detections, ztf_detection_add_dir

## Calculate Direction of the Detections from RA/DEC

In [3]:
# Date range
mjd0: int = 58270
mjd1: int = mjd0 + 10

In [4]:
# Get DataFrame of raw detections
params = {'mjd0':mjd0, 'mjd1': mjd1}
df = sp2df('KS.GetRawDetections', params)

In [5]:
# Calculate direction
ztf_detection_add_dir(df)

In [8]:
df

,DetectionTimeID,DetectionID,mjd,ra,dec,ux,uy,uz,mag
0,1,1,58270.167662,197.497479,9.661561,-0.940203,-0.205183,0.271880,17.028400
1,1,2,58270.167662,197.528147,9.462841,-0.940593,-0.207164,0.269011,20.005800
2,1,3,58270.167662,197.125117,9.456218,-0.942678,-0.201138,0.266274,19.185200
3,1,4,58270.167662,196.874981,9.368209,-0.944177,-0.198029,0.263275,19.778099
4,1,5,58270.167662,196.790613,9.262449,-0.944884,-0.197557,0.261085,19.809299
...,...,...,...,...,...,...,...,...,...
676048,910,676049,58279.457650,328.822659,49.862018,0.551526,-0.002082,0.834155,18.224300
676049,910,676050,58279.457650,328.208625,49.975793,0.546627,-0.006244,0.837353,18.119600
676050,910,676051,58279.457650,329.031648,49.880229,0.552531,-0.000038,0.833492,18.830299
676051,910,676052,58279.457650,328.586834,49.929302,0.549381,-0.003432,0.835565,18.100000


## Get SkyPatch Data

In [6]:
# Get cube faces
cf = sp2df('KS.GetCubeFace')

In [7]:
cf

,CubeFaceID,CubeFaceCD,i,j1,j2,ci,alpha,beta,gamma
0,1,Z+,3,1,2,1.0,X,Y,Z
1,2,Y+,2,3,1,1.0,Z,X,Y
2,3,X+,1,2,3,1.0,Y,Z,X
3,4,X-,1,2,3,-1.0,Y,Z,X
4,5,Y-,2,3,1,-1.0,Z,X,Y
5,6,Z-,3,1,2,-1.0,X,Y,Z


In [78]:
# Get the SkyPatch DataFrame
sp = sp2df('KS.GetSkyPatch')

In [10]:
sp

,SkyPatchID,f,i,j,x,y,z
0,0,0,0,0,-0.577256,-0.577256,0.577538
1,1,0,0,1,-0.577444,-0.576880,0.577726
2,2,0,0,2,-0.577632,-0.576503,0.577914
3,3,0,0,3,-0.577820,-0.576126,0.578102
4,4,0,0,4,-0.578008,-0.575749,0.578290
...,...,...,...,...,...,...,...
25165819,25165819,5,2047,2043,0.578008,0.575749,-0.578290
25165820,25165820,5,2047,2044,0.577820,0.576126,-0.578102
25165821,25165821,5,2047,2045,0.577632,0.576503,-0.577914
25165822,25165822,5,2047,2046,0.577444,0.576880,-0.577726


## Insert batch (Just Directions; SkyPatchID Calculated Later)

In [40]:
columns = ['DetectionTimeID','DetectionID','ux', 'uy', 'uz', 'mag']
df2db(df=df, schema='KS', table='DetectionBatch', columns=columns)

  0%|          | 0/1 [00:00<?, ?it/s]

## Calculate SkyPatchID from the Directions

In [63]:
cf

,CubeFaceID,CubeFaceCD,i,j1,j2,ci,alpha,beta,gamma
0,1,Z+,3,1,2,1.0,X,Y,Z
1,2,Y+,2,3,1,1.0,Z,X,Y
2,3,X+,1,2,3,1.0,Y,Z,X
3,4,X-,1,2,3,-1.0,Y,Z,X
4,5,Y-,2,3,1,-1.0,Z,X,Y
5,6,Z-,3,1,2,-1.0,X,Y,Z


In [83]:
def dir2SkyPatchID(dir: np.array, N: int):
    """
    Compute a vector of SkyPatchIDs from a 3 vectors of direction components
    INPUTS:
        dir: An array [ux, uy, uz] of directions on the unit sphere in the ecliptic frame; shape N_row x 3
        N: Grid size for SkyPatch
    """

    # Unpack components of directions
    x = dir[:,0]
    y = dir[:,1]
    z = dir[:,2]

    # The search array for k is ordered z, y, x, -x, -y, -z
    sa = np.stack([z, y, x, -x, -y, -z]).T

    # The index of of the largest element; zero based.  CubeFaceID = f+1
    f = np.argmax(sa, axis=1)

    # Row indexer for 2D array indexing
    N_row: int = sa.shape[0]
    row_idx = np.arange(N_row)

    # Indices for axes corresponding to u and v and w
    idx_j1 = cf.j1[f].values-1
    idx_j2 = cf.j2[f].values-1
    idx_i = cf.i[f].values-1

    # Index of direction in the regular order
    k = (3-f) % 3

    # The projection of the direction onto the cube
    a = t*dir[row_idx, idx_j1]
    b = t*dir[row_idx, idx_j2]
    c = t*dir[row_idx, idx_i]

    # Compute the integer entries (i, j)
    i = np.floor(N * (1.0 + a))
    j = np.floor(N * (1.0 + b))

    # Caclulate SkyPatchID
    M: int = (2*N)
    M2: int = M*M
    SkyPatchID = f*M2 + i*M + j
    return SkyPatchID

In [84]:
# Nx3 array of directions
dir = df[['ux', 'uy', 'uz']].values

In [86]:
SkyPatchID = dir2SkyPatchID(dir, N)

In [87]:
dir_sp = sp.loc[SkyPatchID, ['x', 'y', 'z']].values

In [88]:
dist = np.linalg.norm(diff, axis=1)

In [92]:
np.max(dist)*360.0

0.24436361252903707

In [93]:
np.mean(dist)*360.0

0.0977585093661368